# Hana Data Labs Pilot - EDA

## Import Seeded Data

In [1]:
import os
import pandas as pd
import numpy as np

## Summarize Data

In [3]:
#--------------------------------------- Data Prep ---------------------------------------
# Change 'order_datetime' column to datetime datatype
def basic_stats(df: pd.DataFrame):
    stats_types = df.info()
    stats_desc = df.describe()

    print(df.columns)
    print(stats_types)
    print(stats_desc)

    return stats_types, stats_desc

## Visualize Data

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import calendar


# ---------------------------------------------------------------------
# |       1. Sales in 2025 by item - histogram                         |
# |       2. Sales throughout year - line chart                        |
# |       3. Progression of qty of best item within each month         |
# ---------------------------------------------------------------------

# ------------------------ Global Plot Style ------------------------
def set_report_style():
    sns.set_theme(
        style="whitegrid",        # light grid, good for reports
        context="talk",           # slightly larger fonts
        palette="muted"           # soft, modern colors
    )

    plt.rcParams.update({
        "figure.figsize": (12, 5),
        "axes.titlesize": 18,
        "axes.labelsize": 14,
        "xtick.labelsize": 11,
        "ytick.labelsize": 11,
        "legend.fontsize": 11,
        "figure.autolayout": True,
        "axes.spines.top": False,
        "axes.spines.right": False,
    })


# visualization function
def visualize(df: pd.DataFrame, year: int):
    # Apply global style
    set_report_style()

    # Filter data for only year == 'year'
    current_year_data = df.loc[df['order_datetime'].dt.year == year].copy()
    
    # Ensure supporting columns exist
    if 'weekday' not in current_year_data.columns:
        current_year_data['weekday'] = current_year_data['order_datetime'].dt.weekday
    
    # Map weekday numbers to names
    weekday_order = [0, 1, 2, 3, 4, 5, 6]
    weekday_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    weekday_map = dict(zip(weekday_order, weekday_labels))
    
    #======================== 1. Sales in 'year' by weekday ========================
    sum_totals_weekday = (
        current_year_data
        .groupby('weekday', as_index=False)['line_total']
        .sum()
    )
    sum_totals_weekday['weekday_name'] = sum_totals_weekday['weekday'].map(weekday_map)

    fig, ax = plt.subplots(figsize=(10, 5))
    sns.barplot(
        data=sum_totals_weekday.sort_values('weekday'),
        x='weekday_name',
        y='line_total',
        ax=ax
    )
    ax.set_xlabel('Day of the Week')
    ax.set_ylabel('Total Amount Sold ($)')
    ax.set_title(f'Total Sales by Day of Week in {year}')

    # Add value labels on top of bars (optional but very “report-y”)
    for container in ax.containers:
        ax.bar_label(container, fmt='${:,.0f}'.format, padding=3)
    
    plt.tight_layout()
    plt.show()
    
    #============================== 2. Sales throughout year ============================== 
    current_year_data['order_date'] = current_year_data['order_datetime'].dt.date
    current_year_data['order_date'] = pd.to_datetime(current_year_data['order_date'])

    sum_totals_daily = (
        current_year_data
        .groupby('order_date', as_index=False)['line_total']
        .sum()
        .rename(columns={'line_total': 'daily_sales'})
    )

    # Optional: add a 7-day rolling average to smooth noise
    sum_totals_daily['roll_7d'] = sum_totals_daily['daily_sales'].rolling(7, center=True).mean()

    fig, ax = plt.subplots(figsize=(12, 5))
    # Raw daily values (lighter)
    sns.lineplot(
        data=sum_totals_daily,
        x='order_date',
        y='daily_sales',
        ax=ax,
        alpha=0.4,
        linewidth=1,
        label='Daily Sales'
    )
    # Smoothed trend
    sns.lineplot(
        data=sum_totals_daily,
        x='order_date',
        y='roll_7d',
        ax=ax,
        linewidth=2.2,
        label='7-Day Rolling Avg'
    )

    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.xaxis.set_minor_locator(mdates.WeekdayLocator(byweekday=mdates.MO))

    ax.set_xlabel('Month')
    ax.set_ylabel('Total Sold ($)')
    ax.set_title(f'Daily Sales and Trend in {year}')
    ax.legend(frameon=True)
    fig.autofmt_xdate()
    plt.tight_layout()
    plt.show()
    
    #===================== 3. Progression of quantity of best grossing item per month =================
    current_year_data['order_month'] = current_year_data['order_datetime'].dt.month

    monthly_item_sales = (
        current_year_data
        .groupby(['order_month', 'item_name'], as_index=False)['line_total']
        .sum()
    )

    # For each month, row with max line_total
    idx = monthly_item_sales.groupby('order_month')['line_total'].idxmax()
    highest_grossing_item_month = (
        monthly_item_sales
        .loc[idx]
        .sort_values('order_month')
        .reset_index(drop=True)
    )
    print("Highest grossing items by month:")
    print(highest_grossing_item_month)

    # Plot each month’s top item as its own small report-ready chart
    for _, row in highest_grossing_item_month.iterrows():
        month = row['order_month']
        top_item = row['item_name']

        mask = (
            (current_year_data['order_month'] == month) &
            (current_year_data['item_name'] == top_item)
        )
        subset = current_year_data.loc[mask].copy()

        if subset.empty:
            continue

        daily_qty = (
            subset
            .groupby('order_date', as_index=False)['quantity']
            .sum()
        )

        fig, ax = plt.subplots(figsize=(10, 4))
        sns.lineplot(
            data=daily_qty,
            x='order_date',
            y='quantity',
            marker='o',
            linewidth=1.8,
            ax=ax
        )

        ax.xaxis.set_major_locator(mdates.DayLocator(interval=3))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
        plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

        ax.set_xlabel('Date')
        ax.set_ylabel('Quantity Sold')
        ax.set_title(
            f"{calendar.month_name[month]} {year} – Daily Quantity of Top Item\n"
            f"Item: {top_item}"
        )
        plt.tight_layout()
        plt.show()
    
    #================== 4. Item co-occurrence heatmap ==================
    matrix = (
        df
        .assign(value=1)
        .pivot_table(
            index='order_id', 
            columns='item_name', 
            values='value',
            aggfunc='max',
            fill_value=0
        )
    )

    co_matrix = matrix.T.dot(matrix)

    # Normalize (row-wise) for association strength
    norm = co_matrix.divide(co_matrix.max(axis=1).replace(0, np.nan), axis=0)
    np.fill_diagonal(norm.values, 0)

    # Optionally, limit to top N items by overall frequency for readability
    top_n = 20
    item_totals = co_matrix.sum(axis=1).sort_values(ascending=False)
    top_items = item_totals.head(top_n).index
    norm_top = norm.loc[top_items, top_items]

    fig, ax = plt.subplots(figsize=(12, 9))
    sns.heatmap(
        norm_top,
        ax=ax,
        cmap="Blues",
        square=True,
        cbar_kws={"label": "Relative Co-Occurrence Strength"},
    )
    ax.set_title("Item Co-Occurrence Heatmap (Top Items)")
    ax.set_xlabel("Item")
    ax.set_ylabel("Item")
    plt.tight_layout()
    plt.show()

    # keep if you want to return things later
    item_count_by_month = monthly_item_sales
    # return item_count_by_month, highest_grossing_item_month

In [9]:
# Import and prepare data for test
os.chdir(r'/Users/cody/Desktop/Projects/hana-pilot-pos-analytics/data/raw')
line_items = pd.read_csv('indian_food_pos_raw.csv')
# orders = pd.read_csv('pos_orders.csv')

# line_items['order_datetime'] = pd.to_datetime(line_items['order_datetime'])
# line_items['year'] = line_items['order_datetime'].dt.year

line_items.head()



,order_id,date,item_name,item_type,item_price,quantity,transaction_amount,transaction_type,received_by,time_of_sale
0,1,07-03-2022,Aalopuri,Fastfood,20,13,260,NaN,Mr.,Night
1,2,8/23/2022,Vadapav,Fastfood,20,15,300,Cash,Mr.,Afternoon
2,3,11/20/2022,Vadapav,Fastfood,20,1,20,Cash,Mr.,Afternoon
3,4,02-03-2023,Sugarcane juice,Beverages,25,6,150,Online,Mr.,Night
4,5,10-02-2022,Sugarcane juice,Beverages,25,8,200,Online,Mr.,Evening


In [10]:
line_items.columns

Index(['order_id', 'date', 'item_name', 'item_type', 'item_price', 'quantity',
       'transaction_amount', 'transaction_type', 'received_by',
       'time_of_sale'],
      dtype='object')

In [ ]:
# Run test
if __name__ == '__main__':
    # item_counts, highest = visualize(line_items, year=2024)
    visualize(line_items, year=2024)